### 필요한 패키지 Import
DataFrame의 내용을 생략없이 출력하려면 아래 주석처리한 부분의 주석을 해제하고 실행

In [1]:
import json
import os
import sys
import urllib.request

from tqdm.auto import tqdm

import numpy as np
import pandas as pd

from collections import Counter

# pd.set_option('display.max_colwidth', None)

### 변수 정의

In [2]:
DATAPATH = "../../data/preprocessed/"
DATAFILE = os.path.join(DATAPATH, f"train_g2p_removed_label_error_revised.csv")
SOURCE_DATA = pd.read_csv(DATAFILE)
data = SOURCE_DATA.copy()

### 역번역 증강을 위한 API KEY, SECRET 정의

In [3]:
with open("./papago_api_id.txt", "r") as f:
    CLIENT_ID_LIST = [line.rstrip() for line in f.readlines()]

with open("./papago_api_secret.txt", "r") as f:
    CLIENT_SECRET_LIST = [line.rstrip() for line in f.readlines()]

### 번역을 수행하는 함수 정의

In [4]:
def translate(sentence, source, target):
    encText = urllib.parse.quote(sentence)
    data = f"source={source}&target={target}&text=" + encText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    for i, (id, secret) in enumerate(zip(CLIENT_ID_LIST, CLIENT_SECRET_LIST)):
        try:
            request.add_header("X-Naver-Client-Id", id)
            request.add_header("X-Naver-Client-Secret", secret)
            response = urllib.request.urlopen(request, data=data.encode("utf-8"))
            response_body = response.read()
            translated_sentence = json.loads(response_body.decode("utf-8"))["message"]["result"]["translatedText"]
            print(f"{i} ID: {id}, SECRET: {secret} is now being used.")
            return translated_sentence
        except:
            pass

In [5]:
translate("번역된 문장을 활용하여 데이터를 증강합니다.", source="ko", target="en")

0 ID: NrWC1mlRvpcbnzNVLxjl, SECRET: HeQfm6ALBE is now being used.


'Use translated sentences to augment your data.'

In [6]:
translate("Use translated sentences to augment your data.", source="en", target="ko")

0 ID: NrWC1mlRvpcbnzNVLxjl, SECRET: HeQfm6ALBE is now being used.


'번역된 문장을 사용하여 데이터를 보강합니다.'

### 역번역을 적용하는 함수 정의

원본 데이터의 `input_text`에 대하여 역번역을 수행하고, \
역번역된 텍스트를 원본 문장 대신 저장하는 함수 정의

In [7]:
def back_translate(text):
    translated = translate(text, source="ko", target="en")
    back_translated = translate(translated, source="en", target="ko")
    return back_translated

정의한 함수를 활용하여 역번역이 정상적으로 작동되며, 컬럼의 내용이 의도한 대로 변경되는지 확인한다.

In [13]:
sample_data = data.iloc[:3]
sample_data

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
0,ynat-v1_train_00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,생활문화,3,IT과학,"{'annotators': ['08', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.30. 오전 10:36
1,ynat-v1_train_00001,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화,3,생활문화,"{'annotators': ['14', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
2,ynat-v1_train_00002,내년부터 국가RD 평가 때 논문건수는 반영 않는다,사회,2,IT과학,"{'annotators': ['14', '13', '10'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.15. 오후 12:00


In [9]:
sample_data_back_translated = sample_data.assign(
    input_text = sample_data["input_text"].apply(back_translate)
)

0 ID: NrWC1mlRvpcbnzNVLxjl, SECRET: HeQfm6ALBE is now being used.
0 ID: NrWC1mlRvpcbnzNVLxjl, SECRET: HeQfm6ALBE is now being used.
0 ID: NrWC1mlRvpcbnzNVLxjl, SECRET: HeQfm6ALBE is now being used.
0 ID: NrWC1mlRvpcbnzNVLxjl, SECRET: HeQfm6ALBE is now being used.
0 ID: NrWC1mlRvpcbnzNVLxjl, SECRET: HeQfm6ALBE is now being used.
0 ID: NrWC1mlRvpcbnzNVLxjl, SECRET: HeQfm6ALBE is now being used.


In [10]:
sample_data_back_translated

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
0,ynat-v1_train_00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,생활문화,3,IT과학,"{'annotators': ['08', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.30. 오전 10:36
1,ynat-v1_train_00001,어버이날은 맑고 흐린 날씨야...남쪽의 옅은 황사,생활문화,3,생활문화,"{'annotators': ['14', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
2,ynat-v1_train_00002,논문 수는 내년부터 국가 RD평가에 반영되지 않는다,사회,2,IT과학,"{'annotators': ['14', '13', '10'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.15. 오후 12:00


In [14]:
sample_data == sample_data_back_translated

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
0,True,True,True,True,True,True,True,True
1,True,False,True,True,True,True,True,True
2,True,False,True,True,True,True,True,True
